### 获取根路径

In [1]:
import sys
import os

# 获取当前工作目录
# Trytorch\test
current_dir = os.getcwd()

# 获取父目录
# Trytorch
project_dir = os.path.dirname(current_dir)

# 现在的路径即为项目根路径,可以找到主包trytorch
sys.path.append(project_dir)

In [2]:
import numpy as np

import trytorch as torch
import trytorch.ops as ops
import trytorch.nn as nn
import trytorch.optim as optim
import trytorch.datas as data
from trytorch.array_device import *

c:\Users\GameDesign\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


### 搭建网络,优化器,损失函数

In [ ]:
batch_size = 100
epochs = 10
device = cpu()

class Model(nn.Module):
    def __init__(self, hidden = 100 , device = device):
        super().__init__()

        self.block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, hidden, device=device),
            nn.ReLU(),
            nn.Linear(hidden, 10, device=device),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.block(x)

net = Model()
optimizer = optim.Adam(net.parameters(),lr=0.001,weight_decay=0.001)
criterion = nn.SoftmaxLoss()

### 如果测试池化层,则执行该模块

In [3]:
batch_size = 200
epochs = 3
device = cpu()

class LeNet5(nn.Module):
    def __init__(self, device=None, dtype="float32"):
        super().__init__()
        # (N, 1, 28, 28) -> (N, 6, 28, 28)
        self.conv1 = nn.Conv(in_channels=1,out_channels=6, kernel_size=5, stride=1, device=device, dtype=dtype)  # (N, 6, 28, 28)
        self.pool1 = nn.MaxPooling2D(kernel_size=2, stride=2, device=device, dtype=dtype)  # (N, 6, 14, 14)
        
        self.conv2 = nn.Conv(6, 16,kernel_size=5, stride=1, device=device, dtype=dtype)  # (N, 16, 14, 14)
        self.pool2 = nn.MaxPooling2D(kernel_size=2, stride=2, device=device, dtype=dtype)  # (N, 16, 7, 7)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 7 * 7, 120, device=device, dtype=dtype)
        self.fc2 = nn.Linear(120, 84, device=device, dtype=dtype)
        self.fc3 = nn.Linear(84, 10, device=device, dtype=dtype)  # 10类分类

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)  
        x = self.conv2(x)
        x = self.pool2(x)   
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

net = LeNet5()
optimizer = optim.Adam(net.parameters(),lr=0.001,weight_decay=0.001)
criterion = nn.SoftmaxLoss()

### dataset 与 dataloader 测试

In [4]:
from pathlib import Path

project_path = Path(project_dir)
print(project_path)

# 训练数据集
train_dataset = data.MNISTDataset(
    project_path / "data" / "MNIST" / "train-images-idx3-ubyte.gz",
    project_path / "data" / "MNIST" / "train-labels-idx1-ubyte.gz"
)

train_dataloader = data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_dataset = data.MNISTDataset(
    project_path / "data" / "MNIST" / "t10k-images-idx3-ubyte.gz",
    project_path / "data" / "MNIST" / "t10k-labels-idx1-ubyte.gz"
)

test_dataloader = data.DataLoader(
    dataset = test_dataset,
    batch_size=batch_size,
    shuffle=True
)


d:\AIExperienments\TryTorch


In [ ]:
import matplotlib.pyplot as plt

# 显示数据集

img, label = train_dataset[42]
# 因为是 (1, 28, 28)，需要 squeeze 去掉 channel 维度
plt.imshow(img.squeeze(), cmap="gray")
plt.title(f"Label: {label}")
plt.axis("off")
plt.show()

# 显示数据加载器
len(train_dataloader), len(test_dataloader)

### 🤗 训练

In [5]:
from tqdm import tqdm

for epoch in range(epochs):
    total_loss = 0
    total_rights = 0
    total_examples = 0
    total_batches = 0

    train_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}', unit='batch')

    for inputs, label in train_bar:
        
        net.train()
        
        optimizer.reset_grad()

        pred = net(inputs)

        loss = criterion(pred, label)

        loss.backward()

        optimizer.step()
        # (batch, features) -> (batch, 1)
        label_pred = np.argmax(pred.numpy(), axis = 1)

        rights = np.equal(label_pred, label.numpy()).sum()

        total_loss += loss.numpy()
        total_rights += rights
        total_batches += 1
        total_examples += inputs.shape[0]

        # 实时更新进度条信息
        avg_loss_so_far = total_loss / total_batches
        avg_accuracy_so_far = total_rights / total_examples
        
        # 更新进度条描述
        train_bar.set_postfix({
            'loss': f'{avg_loss_so_far:.4f}',
            'acc': f'{avg_accuracy_so_far:.4f}'
        })

    avg_loss = total_loss / total_batches
    avg_accuracy = total_rights / total_examples
    print(f"EPOCH {epoch}: {avg_accuracy=}, {avg_loss=}")



Epoch 1/3:   0%|          | 0/300 [00:00<?, ?batch/s]

Epoch 1/3: 100%|██████████| 300/300 [02:30<00:00,  1.99batch/s, loss=0.3057, acc=0.9070]


EPOCH 0: avg_accuracy=0.9069666666666667, avg_loss=0.30573671423594173


Epoch 2/3: 100%|██████████| 300/300 [02:30<00:00,  1.99batch/s, loss=0.1094, acc=0.9665]


EPOCH 1: avg_accuracy=0.9665166666666667, avg_loss=0.10943053931395225


Epoch 3/3: 100%|██████████| 300/300 [02:34<00:00,  1.95batch/s, loss=0.0775, acc=0.9764]

EPOCH 2: avg_accuracy=0.9764, avg_loss=0.07753533507188154


### 🤗 测试

In [6]:
total_loss = 0
total_rights = 0
total_examples = 0
total_batches = 0

with torch.no_grad():
    for inputs, label in test_dataloader:
        
        net.eval()

        pred = net(inputs)

        loss = criterion(pred, label)

        label_pred = np.argmax(pred.numpy(),axis=1)
        
        rights = np.equal(label_pred, label.numpy()).sum()

        total_loss += loss.numpy()
        total_rights += rights
        total_batches += 1
        total_examples += inputs.shape[0]

    avg_loss = total_loss / total_batches
    avg_accuracy = total_rights / total_examples
    print(f"TEST SCORE: {avg_accuracy=}, {avg_loss=}")

TEST SCORE: avg_accuracy=0.9802, avg_loss=0.06294495978355409
